# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
from dotenv import load_dotenv
import os
load_dotenv()
price_data_path = os.getenv("PRICE_DATA")

print(f"PRICE_DATA path: {price_data_path}")

PRICE_DATA path: ../../05_src/data/prices/


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [6]:
import os
from glob import glob

# Write your code below.

directory_path='../../05_src/data/prices'
PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(directory_path, "**/*.0.parquet"),recursive=True)
len(parquet_files)

13078

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [30]:
# Write your code below.
import dask.dataframe as dd

# Read parquet files
dd_px = dd.read_parquet(parquet_files)

# Clean column names (remove spaces)
dd_px.columns = [col.strip() for col in dd_px.columns]

# Verify column names
print("Updated Columns:", dd_px.columns)

# Define metadata explicitly for Dask
meta = {
    "Date": "datetime64[ns]",
    "Adj Close": "float64",
    "Close": "float64",
    "High": "float64",
    "Low": "float64",
    "Open": "float64",
    "Volume": "int64",
    "Year": "int64",
    "Close_lag_1": "float64",
    "Adj_Close_lag_1": "float64",
    "returns": "float64",
    "hi_lo_range": "float64"
}

# Define function to add lag features
def add_lag_features(df):
    df = df.sort_values("Date")  
    df["Close_lag_1"] = df["Close"].shift(1)
    df["Adj_Close_lag_1"] = df["Adj Close"].shift(1) 
    df["returns"] = (df["Adj Close"] / df["Adj_Close_lag_1"]) - 1
    df["hi_lo_range"] = df["High"] - df["Low"]
    return df

# Apply function using map_partitions with explicit meta
dd_feat = dd_px.map_partitions(add_lag_features, meta=meta)

Updated Columns: Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'Year'], dtype='object', name='Price')


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [31]:
# Write your code below.
import pandas as pd
# Compute and display first few rows
df = dd_feat.compute()
df["returns_ma_10"] = df.groupby("ticker")["returns"].rolling(10).mean().reset_index(level=0, drop=True)
print(df.head())  

KeyError: 'ticker'

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
Yes, becayse dask does not support rolling and mean operations efficiently on large datasets.


+ Would it have been better to do it in Dask? Why?
If the dataset is too large for memory, using Dask with map_overlap or dask.array.rolling would be better. Otherwise, converting to Pandas is simpler and more efficient.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.